In [7]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/kaggle/input/supply-chain-dataset-for-risk-assess/DataCoSupplyChainDataset.csv', encoding='latin-1')

# Display the first few rows
print("DataFrame Head:")
print(df.head())

# Display column information and data types
print("\nDataFrame Info:")
df.info()

DataFrame Head:
       Type  Days for shipping (real)  Days for shipment (scheduled)  \
0     DEBIT                         3                              4   
1  TRANSFER                         5                              4   
2      CASH                         4                              4   
3     DEBIT                         3                              4   
4   PAYMENT                         2                              4   

   Benefit per order  Sales per customer   Delivery Status  \
0          91.250000          314.640015  Advance shipping   
1        -249.089996          311.359985     Late delivery   
2        -247.779999          309.720001  Shipping on time   
3          22.860001          304.809998  Advance shipping   
4         134.210007          298.250000  Advance shipping   

   Late_delivery_risk  Category Id   Category Name Customer City  ...  \
0                   0           73  Sporting Goods        Caguas  ...   
1                   1           

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Reload the dataset to ensure a clean slate for the script
df = pd.read_csv('/kaggle/input/supply-chain-dataset-for-risk-assess/DataCoSupplyChainDataset.csv', encoding='latin-1')

# --- Data Cleaning ---

# 1. Handle Missing Values
# Fill 'Customer Lname' with 'Unknown'
df['Customer Lname'] = df['Customer Lname'].fillna('Unknown')

# Fill 'Customer Zipcode' with mode
df['Customer Zipcode'] = df['Customer Zipcode'].fillna(df['Customer Zipcode'].mode()[0])

# Fill 'Order Zipcode' with 0 as it has many missing values, indicating it might not be relevant for most orders
# or the zipcode information is not consistently recorded. 0 is used as a placeholder.
df['Order Zipcode'] = df['Order Zipcode'].fillna(0)

# Drop 'Product Description' as it's entirely empty
df.drop('Product Description', axis=1, inplace=True)

# 2. Convert Dates and Create Time-related Features
# Convert date columns to datetime objects
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], errors='coerce')
df['shipping date (DateOrders)'] = pd.to_datetime(df['shipping date (DateOrders)'], errors='coerce')

# Drop rows where date conversion failed (if any)
df.dropna(subset=['order date (DateOrders)', 'shipping date (DateOrders)'], inplace=True)

# Ensure 'Days for shipping (real)' and 'Days for shipment (scheduled)' are accurate based on date diff
df['delivery_duration_actual'] = (df['shipping date (DateOrders)'] - df['order date (DateOrders)']).dt.days
df['delivery_duration_scheduled'] = df['Days for shipment (scheduled)']

# Calculate shipping delay
df['shipping_delay_days'] = df['delivery_duration_actual'] - df['delivery_duration_scheduled']

# Calculate inter-arrival time (time between consecutive orders)
# Sort by customer and then by order date to calculate inter-arrival time per customer
df.sort_values(by=['Order Customer Id', 'order date (DateOrders)'], inplace=True)
df['inter_arrival_time'] = df.groupby('Order Customer Id')['order date (DateOrders)'].diff().dt.days

# Fill NaN in inter_arrival_time for first orders with 0 or a sensible value (e.g., median of others)
# For now, let's fill with 0, assuming the first order has no prior inter-arrival time in this dataset context.

df['inter_arrival_time'] = df['inter_arrival_time'].fillna(0)
# Ensure inter_arrival_time is non-negative
df['inter_arrival_time'] = df['inter_arrival_time'].apply(lambda x: max(x, 0))


# 3. Normalize Units - No explicit unit normalization needed as per initial inspection,
# but renaming for clarity and consistency.
df.rename(columns={
    'Benefit per order': 'benefit_per_order',
    'Sales per customer': 'sales_per_customer',
    'Order Item Profit Ratio': 'order_item_profit_ratio',
    'Order Profit Per Order': 'order_profit_per_order',
    'Sales': 'sales_total',
    'Order Item Total': 'order_item_total_price',
    'Order Item Discount': 'order_item_discount',
    'Order Item Discount Rate': 'order_item_discount_rate',
    'Order Item Quantity': 'order_item_quantity',
    'Order Item Product Price': 'order_item_product_price'
}, inplace=True)


# --- Output Cleaned CSV ---
cleaned_output_path = 'cleaned_supply_chain_data.csv'
df.to_csv(cleaned_output_path, index=False)
print(f"Cleaned data saved to {cleaned_output_path}")

# --- Basic EDA Charts ---

# Set style for plots
sns.set_style("whitegrid")

# 1. Distribution of Shipping Delay Days
plt.figure(figsize=(10, 6))
sns.histplot(df['shipping_delay_days'], bins=50, kde=True, color='skyblue')
plt.title('Distribution of Shipping Delay Days')
plt.xlabel('Shipping Delay (Days)')
plt.ylabel('Frequency')
plt.savefig('shipping_delay_days_distribution.png')
plt.close()

# 2. Distribution of Order Profit Per Order (as a proxy for cost impact/benefit)
plt.figure(figsize=(10, 6))
sns.histplot(df['order_profit_per_order'], bins=50, kde=True, color='lightcoral')
plt.title('Distribution of Order Profit Per Order')
plt.xlabel('Order Profit')
plt.ylabel('Frequency')
plt.savefig('order_profit_per_order_distribution.png')
plt.close()

# 3. Distribution of Inter-Arrival Time
plt.figure(figsize=(10, 6))
# Filter out 0s if they skew the distribution heavily, or consider their meaning.
# Here, we keep them as they represent first orders.
sns.histplot(df['inter_arrival_time'], bins=50, kde=True, color='lightgreen')
plt.title('Distribution of Inter-Arrival Time (Days)')
plt.xlabel('Inter-Arrival Time (Days)')
plt.ylabel('Frequency')
plt.savefig('inter_arrival_time_distribution.png')
plt.close()

# 4. Count of Delivery Status
plt.figure(figsize=(10, 6))
sns.countplot(data=df, y='Delivery Status', order=df['Delivery Status'].value_counts().index, palette='viridis')
plt.title('Count of Delivery Status')
plt.xlabel('Count')
plt.ylabel('Delivery Status')
plt.savefig('delivery_status_count.png')
plt.close()

# 5. Count of Late_delivery_risk
plt.figure(figsize=(7, 5))
sns.countplot(data=df, x='Late_delivery_risk', palette='plasma')
plt.title('Count of Late Delivery Risk (0=No Risk, 1=Risk)')
plt.xlabel('Late Delivery Risk')
plt.ylabel('Count')
plt.xticks(ticks=[0, 1], labels=['No Risk', 'Risk'])
plt.savefig('late_delivery_risk_count.png')
plt.close()

# print("\nBasic EDA charts generated: shipping_delay_days_distribution.png, order_profit_per_order_distribution.png, inter_arrival_time_distribution.png, delivery_status_count.png, late_delivery_risk_count.png")

# # Display info of cleaned dataframe
# print("\nCleaned DataFrame Info:")
# df.info()

Cleaned data saved to cleaned_supply_chain_data.csv


/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):



Basic EDA charts generated: shipping_delay_days_distribution.png, order_profit_per_order_distribution.png, inter_arrival_time_distribution.png, delivery_status_count.png, late_delivery_risk_count.png

Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 180519 entries, 9138 to 17863
Data columns (total 56 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Type                           180519 non-null  object        
 1   Days for shipping (real)       180519 non-null  int64         
 2   Days for shipment (scheduled)  180519 non-null  int64         
 3   benefit_per_order              180519 non-null  float64       
 4   sales_per_customer             180519 non-null  float64       
 5   Delivery Status                180519 non-null  object        
 6   Late_delivery_risk             180519 non-null  int64         
 7   Category Id                    180519 non-null  i